In [1]:
import numpy as np
import pyfeng as pf
import pandas as pd

## Implement Medvedev and Lorig's methods

In [2]:
class GarchDiffusionModel():
    def __init__(self, S0, K, texp, r, var0, mr, vov, theta, rho):
        self.S0, self.K, self.texp, self.r, self.var0 = S0, K, texp, r, var0,
        self.mr, self.vov, self.theta, self.rho = mr, vov, theta, rho,
        self.rho_star = np.sqrt(1-rho**2)
    def MonteCarlo(self, n, dt, cp = 1 ):  # Milstein method
        St = np.array([S0]*n)
        var_t=np.array([var0]*n)
        for i in range( int(self.texp/dt) ):
            z1=np.random.randn(n)  #to sample Vt
            z2=np.random.randn(n)  #to sample St
            var_t=np.maximum(self.mr*theta*dt+var_t*(1-self.mr*dt+self.vov*np.sqrt(dt)*z1)+1/2*(self.vov**2)*(var_t**2)*(dt*z1**2-dt),0.000)
            St =St + St*self.r*dt+St*np.sqrt(var_t)*np.sqrt(dt)*(self.rho*z1+self.rho_star *z2) +1/2*var_t**2*St*(dt*(self.rho*z1+self.rho_star *z2)**2-dt)  
            ## St =St*np.exp((self.r-0.5*var_t)*dt+np.sqrt(var_t)*np.sqrt(dt)*(self.rho*z1+self.rho_star *z2))
        if cp == 1:
            p = np.maximum(-self.K+St,0)
        elif cp == -1:
            p = np.maximum(self.K-St,0)
        return np.mean(p)
    def Medvedev(self):
        phi = 1;
        sigma = self.var0**0.5;
        x = np.log(self.S0*np.exp(self.r*self.texp)/self.K)
        theta1 = x/sigma/self.texp**0.5;
        a = (self.mr*self.theta/2/sigma-self.mr*sigma/2-self.vov**2*sigma**(4*phi-3)/8);
        b = self.vov*sigma**(2*phi-1)/2;
        bp = (2*phi-1)/2*self.vov*sigma**(2*phi-2);
        I1 = -self.rho/2*b*theta1;
        I2 = (-5/12*self.rho**2*b**2/sigma+1/6*b**2/sigma+1/6*self.rho**2*b*bp)*theta1**2+a/2+self.rho*b*sigma/4+1/24*(self.rho**2)*(b**2)/sigma+1/12*(b**2)/sigma-1/6*(self.rho**2)*b*bp;
        I = sigma+I1*(self.texp**0.5)+I2*self.texp
        return I;
    def Lorig(self):
        S0, K, texp, r, var0, mr, vov, theta, rho=self.S0, self.K, self.texp, self.r, self.var0, self.mr, self.vov, self.theta, self.rho;
        y = np.log(var0)
        sigma0 = np.exp(y/2);
        sigma1 = (np.log(K) - np.log(S0)+r*texp)*vov*rho*np.exp(y/2)/(4*np.exp(y/2)) + (texp*(2*theta*mr - np.exp(y)*(vov**2 + 2*mr) + np.exp(y)*vov*rho*np.exp(y/2)))/(8*np.exp(y/2));
        sigma2 = (np.log(K) - np.log(S0)+r*texp)**2*vov**2*(2*np.exp(y) - 5*rho*np.exp(y) + rho*np.exp(y/2)*rho*np.exp(y/2))/(48*np.exp(y*3/2)) \
                 + texp*(np.log(K) - np.log(S0)+r*texp)*vov*(-np.exp(y)*vov*rho*np.exp(y) - (-2*theta*mr + np.exp(y)*(vov**2 + 2*mr))* rho*np.exp(y) + rho*np.exp(y/2)*(-10*theta*mr + np.exp(y)*(vov**2 + 2*mr) + 2*np.exp(y)*vov*rho*np.exp(y/2)))/(96*np.exp(y*3/2)) \
                 + (1/(384*np.exp(y*3/2)))*texp*(32*np.exp(2*y)*vov**2 - 4*np.exp(3*y)*texp*vov**2 + 5*np.exp(2*y)*texp*vov**4 + 20*np.exp(2*y)*texp*vov**2*mr - 4*np.exp(y)*texp*vov**2*theta*mr + 20*np.exp(2*y)*texp*mr**2 - 8*np.exp(y)*texp*theta*mr**2 - 12*texp*theta**2*mr**2  \
                    + 4*np.exp(y)*(1 + 2*np.exp(y)*texp)*vov**2*rho*np.exp(y) - 2*np.exp(y)*texp*vov*(-2*theta*mr + np.exp(y)*(vov**2 + 2*mr))*rho*np.exp(y/2) - 2*np.exp(y)*vov*rho*np.exp(y/2)*(-2*texp*theta*mr + 5*np.exp(y)*texp*(vov**2 + 2*mr) - (-4 + np.exp(y)*texp)*vov*rho*np.exp(y/2)));
        sigma3 = np.exp(-2*y)*(np.log(K)-np.log(S0)+r*texp)**3*vov**3*(-16*rho*np.exp(y*3/2)-3/2*np.exp(y)*rho*np.exp(y/2) + rho*np.exp(y/2)*(8*np.exp(y) - rho*np.exp(y)/4) + rho*np.exp(y)*(6*rho*np.exp(y/2) - (1/4*rho*np.exp(y/2))))/(-192*np.exp(y/2)) \
                 + (1/(384*np.exp(y/2)))*np.exp(-2*y)*texp*(np.log(K)-np.log(S0)+r*texp)**2*vov**2*(-12*np.exp(y)*theta*mr* + 7*np.exp(y)*vov*rho*np.exp(y/2*3)+3/2*np.exp(2*y)*vov*rho*np.exp(y/2)-(-2*theta*mr+np.exp(y)*(vov**2 + 2*mr))*rho*np.exp(y)/4 \
                    + rho*np.exp(y)*(-7*np.exp(y)*vov**2-14*np.exp(y)*mr*+46*theta*mr - 8*np.exp(y)*vov*rho*np.exp(y/2) + 3/4*np.exp(y)*vov*rho*np.exp(y/2) )+ rho*np.exp(y/2)*(2*(-14*theta*mr+5*np.exp(y)*(vov**2 + 2*mr))*rho*np.exp(y/2)/2 + 3*np.exp(y)*vov*rho*np.exp(y)/4 - 2*(-2*theta* mr + np.exp(y)*(vov**2 + 2*mr))*(rho*np.exp(y/2)/4))) \
                 - (1/(1536*np.exp(y/2)))*np.exp(-2*y)*texp*(np.log(K) - np.log(S0)+r*texp)*vov*(4*np.exp(y)*(9 + 2*np.exp(y)*texp)*vov**2*rho*np.exp(y/2*3)+2*(4*np.exp(2*y)*vov**2 + 3*np.exp(3*y)*texp*vov**2+16*texp*theta**2*mr**2-8*np.exp(y)*texp*theta*mr*(vov**2+2*mr))*rho*np.exp(y/2)/2  \
                   + 4*np.exp(y)*texp*vov*(-2*theta*mr  + np.exp(y)*(vov**2 + 2*mr))*rho*np.exp(y)/4 - 2*(4*texp*theta**2*mr**2 - 4*np.exp(y)*texp*theta*mr*(vov**2 + 2*mr) + np.exp(2*y)*(texp*vov**4 + 4*texp*mr**2 + 4*vov**2*(2 + texp*mr)))*rho*np.exp(y)/4  \
                   -  2*np.exp(y)*vov*rho*np.exp(y)*(10*texp*theta*mr + 3*np.exp(y)*texp*(vov**2 + 2*mr) + 4*(12 + np.exp(y)*texp)*vov*rho*np.exp(y/2)/2 + 3*(-4 + np.exp(y)*texp)*vov*rho*np.exp(y/2)) + rho*np.exp(y/2)*(16*np.exp(2*y)*vov**2 + 3*np.exp(2*y)*texp*vov**4 + 12*np.exp(2*y)*texp*vov**2*mr  \
                   + 20*np.exp(y)*texp*vov**2*theta*mr + 12*np.exp(2*y)*texp*mr**2 + 40*np.exp(y)*texp*theta*mr**2 - 84*texp*theta**2*mr**2 + 32*np.exp(y)*texp*vov*theta*mr*rho*np.exp(y/2)/2 - 6*np.exp(y)*(-4 + np.exp(y)*texp)*vov**2*rho*np.exp(y)/4 + 8*np.exp(y)*texp*vov*(-2*theta*mr + np.exp(y)*(vov**2 + 2*mr))*rho*np.exp(y/2)/4)) \
              + 1/(3072*np.exp(5*y/2))*texp**2*(-48*np.exp(3*y)*vov**4 + 16*np.exp(4*y)*texp*vov**4 - 3*np.exp(3*y)*texp*vov**6 - 96*np.exp(3*y)*vov**2*mr + 32*np.exp(4*y)*texp*vov**2*mr - 18*np.exp(3*y)*texp*vov**4*mr + 64*np.exp(2*y)*vov**2*theta*mr - 8*np.exp(3*y)*texp*vov**2*theta*mr + 2*np.exp(2*y)*texp*vov**4*theta*mr - 36*np.exp(3*y)*texp*vov**2*mr**2 + 8*np.exp(2*y)*texp*vov**2*theta*mr**2 \
                   - 4*np.exp(y)*texp*vov**2*theta**2*mr**2 - 24*np.exp(3*y)*texp*mr**3 + 8*np.exp(2*y)*texp*theta*mr**3 - 8*np.exp(y)*texp*theta**2*mr**3 + 24*texp*theta**3*mr**3 + 2*np.exp(2*y)*(6 + 5*np.exp(y)*texp)*vov**3*rho*np.exp(3*y/2) - 2*np.exp(2*y)*vov*(3*np.exp(2*y)*texp*vov**2 + 8*texp*theta*mr*(vov**2 + 2*mr) - 4*np.exp(y)*(texp*vov**4 + 4*texp*mr**2 + vov**2*(5 + 4*texp*mr)))*rho*np.exp(y/2)/2  \
                   - 2*np.exp(y)*(-4 + np.exp(y)*texp)*vov**2*(-2*theta*mr + np.exp(y)*(vov**2 + 2*mr))*rho*np.exp(y)/4 + 4*np.exp(3*y)*vov**3*rho*np.exp(y/2) + np.exp(3*y)*texp*vov**5*rho*np.exp(y/2)/2 + 2*np.exp(3*y)*texp*vov**3*mr*rho*np.exp(y/2) - 2*np.exp(2*y)*texp*vov**3*theta*mr*rho*np.exp(y/2) + 2*np.exp(3*y)*texp*vov*mr**2*rho*np.exp(y/2)  \
                   - 4*np.exp(2*y)*texp*vov*theta*mr**2*rho*np.exp(y/2) + 2*np.exp(y)*texp*vov*theta**2*mr**2*rho*np.exp(y/2) - 2*np.exp(y)*vov**2*rho*np.exp(y)*(20*theta*mr + 6*np.exp(y)*(vov**2 + 2*mr) + 8*np.exp(2*y)*texp*(vov**2 + 2*mr) - 4*np.exp(2*y)*texp*vov*rho*np.exp(y/2) - np.exp(y)*(-12 + np.exp(y)*texp)*vov*rho*np.exp(y/2)/4) \
                   + np.exp(y)*vov*rho*np.exp(y/2)*(48*np.exp(2*y)*vov**2 - 16*np.exp(3*y)*texp*vov**2 + 9*np.exp(2*y)*texp*vov**4 + 36*np.exp(2*y)*texp*vov**2*mr - 4*np.exp(y)*texp*vov**2*theta*mr + 36*np.exp(2*y)*texp*mr**2 - 8*np.exp(y)*texp*theta*mr**2 + 4*texp*theta**2*mr**2 - 2*vov*(-16*theta*mr - 6*np.exp(y)*texp*theta*mr + 5*np.exp(2*y)*texp*(vov**2 + 2*mr))*rho*np.exp(y/2)  \
                   + np.exp(y)*(-12 + np.exp(y)*texp)*vov**2*rho*np.exp(y/2)/2 - (-4 + np.exp(y)*texp)*vov*(-2*theta*mr + np.exp(y)*(vov**2 + 2*mr))*rho*np.exp(y/2)))
        return sigma0+sigma1+sigma2+sigma3

## Parameters Setting：
$n=1000000, dt=1/252,S0=100,K=[90,110], T=[15,30,60] fixed $

In [39]:
n, dt, S0= 1000000, 1/252, 100

## Case 1 
$$r=0, v_0=0.16/18 , mr=18 , vov=1.8 , \theta=0.16/18 , \rho=0$$

In [31]:
r, var0, mr, vov, theta, rho= 0.0, 0.16/18, 18, 1.8, 0.16/18, 0.0
rho_star = np.sqrt(1-rho**2)

In [40]:
Karray = np.arange(90,111,5);
Tarray = np.array([15,30,60])/252;
dfs = [df1,df2,df3,df4];
for i in range(len(Tarray)):
    IVmc = np.zeros(len(Karray));
    IVme = np.zeros(len(Karray));
    IVlo = np.zeros(len(Karray));
    error1 = np.zeros(len(Karray));
    error2 = np.zeros(len(Karray));
    for j in range(len(Karray)):
        m = GarchDiffusionModel(S0, Karray[j], Tarray[i], r, var0, mr, vov, theta, rho)
        pmc = m.MonteCarlo(n,dt,1)
        IVme[j] = m.Medvedev()
        IVlo[j] = m.Lorig()
            #p_me = pf.Bsm(sigma = IVme, intr= r , divr=0.0)
            #p_lo = pf.Bsm(sigma = IVlo, intr= r , divr=0.0)
        bsm = pf.Bsm(sigma = 0.02, intr= r , divr=0.0)
        IVmc[j] = bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
        error1[j] = m.Medvedev()-bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
        error2[j] = m.Lorig()-bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
    dfs[i] = pd.DataFrame([IVmc,IVme,IVlo,error1,error2],columns=Karray,index=["IVmc","IVMed","IVLorig","error1","error2"],dtype=float)

In [41]:
print(dfs[0])
print(dfs[1])
print(dfs[2])

              90        95        100       105       110
IVmc     0.141134  0.097095  0.093740  0.095864  0.097942
IVMed    0.109418  0.097291  0.093523  0.096932  0.106531
IVLorig  0.109780  0.097653  0.093885  0.097294  0.106893
error1  -0.031716  0.000196 -0.000217  0.001068  0.008588
error2  -0.031354  0.000558  0.000145  0.001430  0.008950
              90        95        100       105       110
IVmc     0.112679  0.094328  0.093768  0.094690  0.097231
IVMed    0.108661  0.096533  0.092766  0.096174  0.105773
IVLorig  0.109159  0.097031  0.093263  0.096672  0.106271
error1  -0.004018  0.002205 -0.001002  0.001485  0.008542
error2  -0.003520  0.002703 -0.000505  0.001982  0.009040
              90        95        100       105       110
IVmc     0.099634  0.094556  0.094028  0.094330  0.095289
IVMed    0.107146  0.095018  0.091250  0.094659  0.104258
IVLorig  0.101534  0.089407  0.085639  0.089048  0.098647
error1   0.007512  0.000461 -0.002778  0.000329  0.008969
error2   0.001

## Case 2
$$r=0, v_0=0.53/29.23 , mr=29.23 , vov=3.65 , \theta=0.53/29.23 , \rho=0$$

In [42]:
r, var0, mr, vov, theta, rho= 0.0, 0.53/29.23, 29.23, 3.65, 0.53/29.23, 0.0
rho_star = np.sqrt(1-rho**2)

In [43]:
Karray = np.arange(90,111,5);
Tarray = np.array([15,30,60])/252;
dfs = [df1,df2,df3,df4];
for i in range(len(Tarray)):
    IVmc = np.zeros(len(Karray));
    IVme = np.zeros(len(Karray));
    IVlo = np.zeros(len(Karray));
    error1 = np.zeros(len(Karray));
    error2 = np.zeros(len(Karray));
    for j in range(len(Karray)):
        m = GarchDiffusionModel(S0, Karray[j], Tarray[i], r, var0, mr, vov, theta, rho)
        pmc = m.MonteCarlo(n,dt,1)
        IVme[j] = m.Medvedev()
        IVlo[j] = m.Lorig()
            #p_me = pf.Bsm(sigma = IVme, intr= r , divr=0.0)
            #p_lo = pf.Bsm(sigma = IVlo, intr= r , divr=0.0)
        bsm = pf.Bsm(sigma = 0.02, intr= r , divr=0.0)
        IVmc[j] = bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
        error1[j] = m.Medvedev()-bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
        error2[j] = m.Lorig()-bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
    dfs[i] = pd.DataFrame([IVmc,IVme,IVlo,error1,error2],columns=Karray,index=["IVmc","IVMed","IVLorig","error1","error2"],dtype=float)

In [44]:
print(dfs[0])
print(dfs[1])
print(dfs[2])

              90        95        100       105       110
IVmc     0.157729  0.137647  0.133106  0.136276  0.145041
IVMed    0.175968  0.141052  0.130206  0.140019  0.167654
IVLorig  0.177354  0.142438  0.131592  0.141405  0.169040
error1   0.018240  0.003405 -0.002900  0.003744  0.022613
error2   0.019626  0.004791 -0.001514  0.005130  0.024000
              90        95        100       105       110
IVmc     0.140864  0.136051  0.133170  0.134647  0.138451
IVMed    0.171519  0.136603  0.125757  0.135570  0.163205
IVLorig  0.160271  0.125355  0.114509  0.124322  0.151957
error1   0.030655  0.000552 -0.007414  0.000923  0.024754
error2   0.019407 -0.010696 -0.018662 -0.010325  0.013506
              90        95        100       105       110
IVmc     0.139024  0.134881  0.133739  0.134028  0.135489
IVMed    0.162620  0.127704  0.116858  0.126672  0.154306
IVLorig -0.016710 -0.051626 -0.062472 -0.052659 -0.025024
error1   0.023597 -0.007176 -0.016881 -0.007357  0.018817
error2  -0.155

## Case 3
$$r=0, v_0=0.18/2 , mr=2 , vov=0.8 , \theta=0.18/2 , \rho=0$$

In [46]:
r, var0, mr, vov, theta, rho= 0.0, 0.18/2, 2, 0.8, 0.18/2, 0.0
rho_star = np.sqrt(1-rho**2)

In [47]:
Karray = np.arange(90,111,5);
Tarray = np.array([15,30,60])/252;
dfs = [df1,df2,df3,df4];
for i in range(len(Tarray)):
    IVmc = np.zeros(len(Karray));
    IVme = np.zeros(len(Karray));
    IVlo = np.zeros(len(Karray));
    error1 = np.zeros(len(Karray));
    error2 = np.zeros(len(Karray));
    for j in range(len(Karray)):
        m = GarchDiffusionModel(S0, Karray[j], Tarray[i], r, var0, mr, vov, theta, rho)
        pmc = m.MonteCarlo(n,dt,1)
        IVme[j] = m.Medvedev()
        IVlo[j] = m.Lorig()
            #p_me = pf.Bsm(sigma = IVme, intr= r , divr=0.0)
            #p_lo = pf.Bsm(sigma = IVlo, intr= r , divr=0.0)
        bsm = pf.Bsm(sigma = 0.02, intr= r , divr=0.0)
        IVmc[j] = bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
        error1[j] = m.Medvedev()-bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
        error2[j] = m.Lorig()-bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
    dfs[i] = pd.DataFrame([IVmc,IVme,IVlo,error1,error2],columns=Karray,index=["IVmc","IVMed","IVLorig","error1","error2"],dtype=float)

In [48]:
print(dfs[0])
print(dfs[1])
print(dfs[2])

              90        95        100       105       110
IVmc     0.301816  0.301729  0.299714  0.299555  0.299808
IVMed    0.300511  0.299758  0.299524  0.299735  0.300331
IVLorig  0.300549  0.299797  0.299563  0.299774  0.300370
error1  -0.001306 -0.001971 -0.000190  0.000181  0.000524
error2  -0.001267 -0.001932 -0.000151  0.000220  0.000563
              90        95        100       105       110
IVmc     0.300177  0.299182  0.299199  0.299743  0.299425
IVMed    0.300034  0.299281  0.299048  0.299259  0.299855
IVLorig  0.300182  0.299429  0.299195  0.299407  0.300003
error1  -0.000143  0.000100 -0.000151 -0.000484  0.000430
error2   0.000005  0.000247 -0.000003 -0.000336  0.000578
              90        95        100       105       110
IVmc     0.298374  0.298421  0.298255  0.298468  0.298799
IVMed    0.299082  0.298329  0.298095  0.298307  0.298903
IVLorig  0.299612  0.298859  0.298625  0.298837  0.299433
error1   0.000708 -0.000092 -0.000159 -0.000161  0.000104
error2   0.001

## Case 4
$$r=0, v_0=0.18/2 , mr=2 , vov=1.2 , \theta=0.18/2 , \rho=0$$

In [49]:
r, var0, mr, vov, theta, rho= 0.0, 0.18/2, 2, 1.2, 0.18/2, 0.0
rho_star = np.sqrt(1-rho**2)

In [50]:
Karray = np.arange(90,111,5);
Tarray = np.array([15,30,60])/252;
dfs = [df1,df2,df3,df4];
for i in range(len(Tarray)):
    IVmc = np.zeros(len(Karray));
    IVme = np.zeros(len(Karray));
    IVlo = np.zeros(len(Karray));
    error1 = np.zeros(len(Karray));
    error2 = np.zeros(len(Karray));
    for j in range(len(Karray)):
        m = GarchDiffusionModel(S0, Karray[j], Tarray[i], r, var0, mr, vov, theta, rho)
        pmc = m.MonteCarlo(n,dt,1)
        IVme[j] = m.Medvedev()
        IVlo[j] = m.Lorig()
            #p_me = pf.Bsm(sigma = IVme, intr= r , divr=0.0)
            #p_lo = pf.Bsm(sigma = IVlo, intr= r , divr=0.0)
        bsm = pf.Bsm(sigma = 0.02, intr= r , divr=0.0)
        IVmc[j] = bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
        error1[j] = m.Medvedev()-bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
        error2[j] = m.Lorig()-bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
    dfs[i] = pd.DataFrame([IVmc,IVme,IVlo,error1,error2],columns=Karray,index=["IVmc","IVMed","IVLorig","error1","error2"],dtype=float)

In [51]:
print(dfs[0])
print(dfs[1])
print(dfs[2])

              90        95        100       105       110
IVmc     0.304413  0.301813  0.298655  0.299909  0.300314
IVMed    0.301149  0.299455  0.298929  0.299405  0.300745
IVLorig  0.301232  0.299538  0.299012  0.299488  0.300829
error1  -0.003265 -0.002358  0.000274 -0.000504  0.000431
error2  -0.003181 -0.002275  0.000357 -0.000421  0.000515
              90        95        100       105       110
IVmc     0.301251  0.298120  0.297692  0.298872  0.299333
IVMed    0.300077  0.298383  0.297857  0.298333  0.299674
IVLorig  0.300390  0.298696  0.298170  0.298646  0.299987
error1  -0.001173  0.000264  0.000165 -0.000539  0.000341
error2  -0.000861  0.000576  0.000478 -0.000226  0.000653
              90        95        100       105       110
IVmc     0.298233  0.297055  0.297439  0.297978  0.297629
IVMed    0.297934  0.296240  0.295714  0.296190  0.297531
IVLorig  0.299020  0.297326  0.296800  0.297276  0.298617
error1  -0.000298 -0.000814 -0.001725 -0.001788 -0.000098
error2   0.000

## Case 5
$$r=0, v_0=0.09/4 , mr=4 , vov=1.2 , \theta=0.09/4 , \rho=0$$

In [52]:
r, var0, mr, vov, theta, rho= 0.0, 0.09/4, 4, 1.2, 0.09/4, 0.0
rho_star = np.sqrt(1-rho**2)

In [53]:
Karray = np.arange(90,111,5);
Tarray = np.array([15,30,60])/252;
dfs = [df1,df2,df3,df4];
for i in range(len(Tarray)):
    IVmc = np.zeros(len(Karray));
    IVme = np.zeros(len(Karray));
    IVlo = np.zeros(len(Karray));
    error1 = np.zeros(len(Karray));
    error2 = np.zeros(len(Karray));
    for j in range(len(Karray)):
        m = GarchDiffusionModel(S0, Karray[j], Tarray[i], r, var0, mr, vov, theta, rho)
        pmc = m.MonteCarlo(n,dt,1)
        IVme[j] = m.Medvedev()
        IVlo[j] = m.Lorig()
            #p_me = pf.Bsm(sigma = IVme, intr= r , divr=0.0)
            #p_lo = pf.Bsm(sigma = IVlo, intr= r , divr=0.0)
        bsm = pf.Bsm(sigma = 0.02, intr= r , divr=0.0)
        IVmc[j] = bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
        error1[j] = m.Medvedev()-bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
        error2[j] = m.Lorig()-bsm.impvol(pmc, Karray[j], S0, Tarray[i],1)
    dfs[i] = pd.DataFrame([IVmc,IVme,IVlo,error1,error2],columns=Karray,index=["IVmc","IVMed","IVLorig","error1","error2"],dtype=float)

In [54]:
print(dfs[0])
print(dfs[1])
print(dfs[2])

              90        95        100       105       110
IVmc     0.164147  0.152388  0.149835  0.150166  0.151601
IVMed    0.153905  0.150517  0.149464  0.150416  0.153098
IVLorig  0.153988  0.150600  0.149548  0.150500  0.153182
error1  -0.010243 -0.001872 -0.000370  0.000251  0.001496
error2  -0.010159 -0.001788 -0.000287  0.000335  0.001580
              90        95        100       105       110
IVmc     0.147877  0.150341  0.148939  0.149829  0.151568
IVMed    0.153369  0.149981  0.148929  0.149881  0.152562
IVLorig  0.153667  0.150279  0.149227  0.150179  0.152861
error1   0.005492 -0.000360 -0.000010  0.000052  0.000994
error2   0.005790 -0.000062  0.000288  0.000350  0.001292
              90        95        100       105       110
IVmc     0.149811  0.149380  0.148778  0.148973  0.150414
IVMed    0.152297  0.148910  0.147857  0.148809  0.151491
IVLorig  0.153203  0.149815  0.148762  0.149714  0.152396
error1   0.002487 -0.000470 -0.000921 -0.000163  0.001076
error2   0.003